In [1]:
##data load ### cpu 버전

import os
import pickle
import tensorflow as tf
import glob
import numpy as np
import matplotlib.pyplot as plt
import time
from data_cpu import Dataset, LMOrderedIterator
from model import TFTransfoXLModel,TFTransfoXLLMHeadModel

from transformers import TransfoXLConfig
tf.compat.v1.enable_eager_execution()


config_xl = TransfoXLConfig(
    data = '/home/jun/workspace/wiki_short/',
    dataset = 'wt103',
    d_embed=128,
    d_head = 2,
    d_model=10,
    mem_len=5,
    n_head=2,
    n_layer=6,
    batch_size = 2,
    tgt_len = 3,
    ext_len = 0,
    eval_tgt_len = 36
)



kwargs = {}
if config_xl.dataset in ['wt103', 'wt2']:
    kwargs['special'] = ['<eos>','<UNK>']
    kwargs['lower_case'] = False

dataset = Dataset(**kwargs)

train_dataset, val_dataset, test_dataset = dataset.make_dataset(config_xl.data,config_xl.dataset)

# strategy = tf.distribute.MirroredStrategy()

2023-11-16 18:26:50.192175: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-16 18:26:50.746766: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2023-11-16 18:26:50.746809: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.4/lib64:
2023-11-16 18:26:50.746814: W tensorflow/compiler/tf2tensorrt/utils/py_uti

Producing dataset wt103...
final vocab size 109697 from 109695 unique tokens


In [2]:
400 // 10*1

40

분산처리를 하기위해서는 train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
이 코드를 실행해야하는데ㅐ 기본 텐서 구조는 안되더라 그래서 텐서 슬라이스 변경해서 이 구조로 바꾸고 데이터를 이터레이션 해서 모델에 집어 넣어야 하는 방식을 고민중 


In [3]:
def gen(data,bsz,bptt,ext_len=None,):
  
  bsz = bsz#3 #60
  bptt = bptt#36 #70
  ext_len = ext_len if ext_len is not None else 0
  data = data
  
  
  # Work out how cleanly we can divide the dataset into bsz parts.
  # 아래의 두 코드는   data 텐서에서 배치 크기 bsz로 깔끔하게 맞지 않는 추가 요소를 제거하는 것 배치에 띡 떨어지게
  n_step = len(data) // (bsz*bptt)
  print('n_step',n_step) # 312779
  
  sliced_data = tf.slice(data,[0],[n_step * bsz*bptt])  
  # print('sliced_data',len(sliced_data))
  # sliced_data = self.data[:self.n_step * self.bsz]
  '''# 시작 위치와 슬라이싱할 크기 설정
  begin = [0]  # 첫 번째 차원의 시작 위치는 0
  size = [6]   # 첫 번째 차원에서 6개의 원소를 슬라이싱

  # 데이터를 잘라내기 (tf.slice 사용)
  sliced_data = tf.slice(data, begin, size)  '''

  # Evenly divide the da
  # ta across the bsz batches.


  new_shape = (bsz, -1)  # 나머지 차원은 자동으로 계산됨
  data_reshaped = tf.reshape(sliced_data, new_shape)
  # data_transposed = tf.transpose(data_reshaped)
  data = data_reshaped
  # print('data',len(data))
  split_num = 2 #GPU num


  first_half, second_half = tf.split(data, num_or_size_splits=split_num, axis=1)

  n_batch = (n_step + bptt - 1) // bptt

  for i in range(0, len(first_half[1]) - 1, bptt):
    
    seq_len = min(bptt, first_half.shape[1] - 1 - i) # # i값이 103227020를 넘지 않는 이상 seq_len = 70


    end_idx = i + seq_len # 70,71,72,73,74......
    beg_idx = max(0, i - ext_len) # 0,1,2,3,4,5
    ''' 아래 처럼 첫번째 차원을 자르는 이류
    로,또,1,등,당,첨 = > 로,또,1    => 로, 등
                    등,당,첨         또, 당
                                    1, 첨
    '''

    first_half_data = first_half[:,beg_idx:end_idx] # self.data[:,0:70],[:,1:71] ~
    first_half_target = first_half[:,i+1:i+1+seq_len]

    second_half_data = second_half[:,beg_idx:end_idx] # self.data[:,0:70],[:,1:71] ~
    second_half_target = second_half[:,i+1:i+1+seq_len]
    if i + bptt < len(first_half[1]) - 1:
      yield first_half_data, first_half_target
      yield second_half_data, second_half_target


In [4]:
dataset = tf.data.Dataset.from_generator(
     gen,
     output_signature=(
         tf.TensorSpec(shape=None, dtype=tf.int32),
         tf.TensorSpec(shape=None, dtype=tf.int32),
         ),
     args=(train_dataset,config_xl.batch_size,config_xl.tgt_len)
         )

2023-11-16 18:26:55.562153: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-16 18:26:55.562476: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-16 18:26:55.566685: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-16 18:26:55.567021: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-16 18:26:55.567306: I tensorflow/compiler/xla/stream_executo

In [5]:
# sample=next(iter(dataset))
# print(sample)

In [6]:
strategy = tf.distribute.MirroredStrategy()

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

BATCH_SIZE_PER_REPLICA = 1
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

EPOCHS = 2

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


In [7]:
train_dist_dataset = strategy.experimental_distribute_dataset(dataset)


2023-11-16 18:26:56.500579: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\017TensorDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_INT32
        }
      }
    }
  }
}



In [8]:
batched_dataset =dataset.batch(GLOBAL_BATCH_SIZE)
batched_dataset.cache().prefetch(tf.data.AUTOTUNE)

<PrefetchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.int32, name=None), TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))>

In [9]:
# count =0 
# for sample in batched_dataset:
#     # print(sample[0].shape)
#     # print(sample[1].shape)
#     count += 1
# # print(count)

In [10]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = tf.cast(d_model, tf.float32)

    self.warmup_steps = tf.cast(warmup_steps,tf.float32)

  def __call__(self, step):
    step =tf.cast(step, tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [11]:
learning_rate = CustomSchedule(config_xl.d_model)


In [12]:
with strategy.scope():
  def compute_loss(loss):
    
    return tf.nn.compute_average_loss(loss, global_batch_size=GLOBAL_BATCH_SIZE)

In [13]:
with strategy.scope():

    model = TFTransfoXLLMHeadModel(config=config_xl)
    optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)
    checkpoint_path = "./checkpoints/train"

    ckpt = tf.train.Checkpoint(transformer=model,
                            optimizer=optimizer)

    ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

    # if a checkpoint exists, restore the latest checkpoint.
    if ckpt_manager.latest_checkpoint:
        ckpt.restore(ckpt_manager.latest_checkpoint)
        print('Latest checkpoint restored!!')

In [14]:
def train_step(data,target,mems):


  with tf.GradientTape() as tape:
    outputs = model(input_ids=data,labels=target,mems=mems)
    loss = compute_loss(outputs.loss)
    mems = outputs.mems

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  return mems,loss

In [15]:
# `run` replicates the provided computation and runs it
# with the distributed input.
@tf.function
def distributed_train_step(dataset_inputs,target,mems):
  mems,per_replica_losses = strategy.run(train_step, args=(dataset_inputs,target,mems))
  return mems,strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                         axis=None)

In [16]:

for epoch in range(EPOCHS):
  start = time.time()

  mems = None
  total_loss = 0.0
  num_batches = 0
  with tf.profiler.experimental.Trace('train', step_num=epoch, _r=1):
    for x in train_dist_dataset:
      data,target=x
      mems,loss = distributed_train_step(data,target,mems)
      total_loss += loss.numpy()
      num_batches += 1
      
      if num_batches % 50 == 0:
        print(f'Epoch {epoch + 1} Batch {num_batches} Loss {loss}')
      
    train_loss = total_loss / num_batches
      
      
    template = ("Epoch {}, Loss: {} ")
    print(template.format(epoch + 1, train_loss,
                          )) 


n_step 938338
rw_head_q (3, 1, 2, 2)
w_head_k (8, 1, 2, 2)
Eager execution: False
AC Tensor("tf_transfo_xllm_head_model/transformer/layers_._0/dec_attn/einsum/Einsum:0", shape=(3, 8, 1, 2), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:0)
r_head_k (8, 2, 2)
BD Tensor("tf_transfo_xllm_head_model/transformer/layers_._0/dec_attn/einsum_1/Einsum:0", shape=(3, 8, 1, 2), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:0)
BD_rel_shift Tensor("tf_transfo_xllm_head_model/transformer/layers_._0/dec_attn/Reshape_5:0", shape=(3, 8, 1, 2), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:0)
rw_head_q (3, 1, 2, 2)
w_head_k (8, 1, 2, 2)
Eager execution: False
AC Tensor("tf_transfo_xllm_head_model/transformer/layers_._1/dec_attn/einsum/Einsum:0", shape=(3, 8, 1, 2), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:0)
r_head_k (8, 2, 2)
BD Tensor("tf_transfo_xllm_head_model/transformer/layers_._1/dec_attn/einsum_1/Einsum:0", shape=(3, 

KeyboardInterrupt: 

In [9]:
import tensorflow as tf

# Define the _rel_shift function
def _rel_shift(x):
    x_size = x.shape.as_list()

    x = tf.pad(x, [[0, 0], [1, 0], [0, 0], [0, 0]])
    x = tf.reshape(x, [x_size[1] + 1, x_size[0], x_size[2], x_size[3]])
    x = tf.slice(x, [1, 0, 0, 0], [-1, -1, -1, -1])
    x = tf.reshape(x, x_size)

    return x
def _rel_shift(self, x):
        x_size = shape_list(x)

        x = tf.pad(x, [[0, 0], [1, 0], [0, 0], [0, 0]])
        x = tf.reshape(x, [x_size[1] + 1, x_size[0], x_size[2], x_size[3]])
        x = tf.slice(x, [1, 0, 0, 0], [-1, -1, -1, -1])
        x = tf.reshape(x, x_size)

        return x

# Create a sample tensor x with shape (3, 8, 1, 2)
x = tf.constant([[[[1, 2]], [[3, 4]], [[5, 6]], [[7, 8]], [[9, 10]], [[11, 12]], [[13, 14]], [[15, 16]]],
                 [[[17, 18]], [[19, 20]], [[21, 22]], [[23, 24]], [[25, 26]], [[27, 28]], [[29, 30]], [[31, 32]]],
                 [[[33, 34]], [[35, 36]], [[37, 38]], [[39, 40]], [[41, 42]], [[43, 44]], [[45, 46]], [[47, 48]]]])

# Apply the _rel_shift function to x
result = _rel_shift(x)

# Print the original and result tensors
print("Original x:")
print(x.numpy())
print("\nResult after applying _rel_shift:")
print(result.numpy())

Original x:
[[[[ 1  2]]

  [[ 3  4]]

  [[ 5  6]]

  [[ 7  8]]

  [[ 9 10]]

  [[11 12]]

  [[13 14]]

  [[15 16]]]


 [[[17 18]]

  [[19 20]]

  [[21 22]]

  [[23 24]]

  [[25 26]]

  [[27 28]]

  [[29 30]]

  [[31 32]]]


 [[[33 34]]

  [[35 36]]

  [[37 38]]

  [[39 40]]

  [[41 42]]

  [[43 44]]

  [[45 46]]

  [[47 48]]]]

Result after applying _rel_shift:
[[[[ 5  6]]

  [[ 7  8]]

  [[ 9 10]]

  [[11 12]]

  [[13 14]]

  [[15 16]]

  [[ 0  0]]

  [[17 18]]]


 [[[19 20]]

  [[21 22]]

  [[23 24]]

  [[25 26]]

  [[27 28]]

  [[29 30]]

  [[31 32]]

  [[ 0  0]]]


 [[[33 34]]

  [[35 36]]

  [[37 38]]

  [[39 40]]

  [[41 42]]

  [[43 44]]

  [[45 46]]

  [[47 48]]]]


In [36]:
import tensorflow as tf



def _rel_shift(x):
  x_size = tf.shape(x)

  x = tf.pad(x, [[0, 0], [1, 0], [0, 0], [0, 0]])
  x = tf.reshape(x, [x_size[1] + 1, x_size[0], x_size[2], x_size[3]])
  x = tf.slice(x, [1, 0, 0, 0], [-1, -1, -1, -1])
  x = tf.reshape(x, x_size)

  return x


# Define tensors based on your scenario
rr_head_q = tf.constant([[[[6, 6], [6, 6]]], [[[7, 7], [7, 7]]], [[[8, 8], [8, 8]]]], dtype=tf.float32)  # Shape: (3,1,2,2)
rw_head_q = tf.constant([[[[6, 6], [6, 6]]], [[[7, 7], [7, 7]]], [[[8, 8], [8, 8]]]], dtype=tf.float32)  # Shape: (3,1,2,2)
w_head_k = tf.constant([[[[1, 1], [1, 1]]], [[[2, 2], [2, 2]]], [[[3, 3], [3, 3]]], [[[4, 4], [4, 4]]], [[[5, 5], [5, 5]]], [[[6, 6], [6, 6]]], [[[7, 7], [7, 7]]], [[[8, 8], [8, 8]]]], dtype=tf.float32)  # Shape: (8,1,2,2)
r_head_k = tf.constant([[[1.1, 1.2], [1.3, 1.4]], [[2.1, 2.2], [2.3, 2.4]], [[3.1, 3.2], [3.3, 3.4]], [[4.1, 4.2], [4.3, 4.4]], [[5.1, 5.2], [5.3, 5.4]], [[6.1, 6.2], [6.3, 6.4]], [[7.1, 7.2], [7.3, 7.4]], [[8.1, 8.2], [8.3, 8.4]]], dtype=tf.float32)  # Shape: (8,2,2)


In [37]:
r_head_k.shape

TensorShape([8, 2, 2])

In [38]:

# Compute AC
AC = tf.einsum("ibnd,jbnd->ijbn", rw_head_q, w_head_k)



In [39]:
AC

<tf.Tensor: shape=(3, 8, 1, 2), dtype=float32, numpy=
array([[[[ 12.,  12.]],

        [[ 24.,  24.]],

        [[ 36.,  36.]],

        [[ 48.,  48.]],

        [[ 60.,  60.]],

        [[ 72.,  72.]],

        [[ 84.,  84.]],

        [[ 96.,  96.]]],


       [[[ 14.,  14.]],

        [[ 28.,  28.]],

        [[ 42.,  42.]],

        [[ 56.,  56.]],

        [[ 70.,  70.]],

        [[ 84.,  84.]],

        [[ 98.,  98.]],

        [[112., 112.]]],


       [[[ 16.,  16.]],

        [[ 32.,  32.]],

        [[ 48.,  48.]],

        [[ 64.,  64.]],

        [[ 80.,  80.]],

        [[ 96.,  96.]],

        [[112., 112.]],

        [[128., 128.]]]], dtype=float32)>

In [40]:

# Compute BD
BD = tf.einsum("ibnd,jnd->ijbn", rr_head_q, r_head_k)

In [41]:
BD

<tf.Tensor: shape=(3, 8, 1, 2), dtype=float32, numpy=
array([[[[ 13.800001,  16.199999]],

        [[ 25.8     ,  28.2     ]],

        [[ 37.8     ,  40.2     ]],

        [[ 49.799995,  52.200005]],

        [[ 61.799995,  64.200005]],

        [[ 73.799995,  76.200005]],

        [[ 85.799995,  88.200005]],

        [[ 97.8     , 100.2     ]]],


       [[[ 16.1     ,  18.9     ]],

        [[ 30.099998,  32.9     ]],

        [[ 44.1     ,  46.9     ]],

        [[ 58.1     ,  60.9     ]],

        [[ 72.1     ,  74.9     ]],

        [[ 86.1     ,  88.9     ]],

        [[100.1     , 102.9     ]],

        [[114.100006, 116.899994]]],


       [[[ 18.400002,  21.599998]],

        [[ 34.4     ,  37.6     ]],

        [[ 50.4     ,  53.6     ]],

        [[ 66.399994,  69.600006]],

        [[ 82.399994,  85.600006]],

        [[ 98.399994, 101.600006]],

        [[114.399994, 117.600006]],

        [[130.4     , 133.6     ]]]], dtype=float32)>

In [42]:


# Apply _rel_shift to BD
BD_shifted = _rel_shift(BD)
BD_shifted

<tf.Tensor: shape=(3, 8, 1, 2), dtype=float32, numpy=
array([[[[ 37.8     ,  40.2     ]],

        [[ 49.799995,  52.200005]],

        [[ 61.799995,  64.200005]],

        [[ 73.799995,  76.200005]],

        [[ 85.799995,  88.200005]],

        [[ 97.8     , 100.2     ]],

        [[  0.      ,   0.      ]],

        [[ 16.1     ,  18.9     ]]],


       [[[ 30.099998,  32.9     ]],

        [[ 44.1     ,  46.9     ]],

        [[ 58.1     ,  60.9     ]],

        [[ 72.1     ,  74.9     ]],

        [[ 86.1     ,  88.9     ]],

        [[100.1     , 102.9     ]],

        [[114.100006, 116.899994]],

        [[  0.      ,   0.      ]]],


       [[[ 18.400002,  21.599998]],

        [[ 34.4     ,  37.6     ]],

        [[ 50.4     ,  53.6     ]],

        [[ 66.399994,  69.600006]],

        [[ 82.399994,  85.600006]],

        [[ 98.399994, 101.600006]],

        [[114.399994, 117.600006]],

        [[130.4     , 133.6     ]]]], dtype=float32)>

In [43]:

# Calculate the final attention score
attn_score = AC + BD_shifted

print('AC:', AC.numpy())
print('BD before shift:', BD.numpy())
print('BD after shift:', BD_shifted.numpy())
print('Attention Score:', attn_score.numpy())

AC: [[[[ 12.  12.]]

  [[ 24.  24.]]

  [[ 36.  36.]]

  [[ 48.  48.]]

  [[ 60.  60.]]

  [[ 72.  72.]]

  [[ 84.  84.]]

  [[ 96.  96.]]]


 [[[ 14.  14.]]

  [[ 28.  28.]]

  [[ 42.  42.]]

  [[ 56.  56.]]

  [[ 70.  70.]]

  [[ 84.  84.]]

  [[ 98.  98.]]

  [[112. 112.]]]


 [[[ 16.  16.]]

  [[ 32.  32.]]

  [[ 48.  48.]]

  [[ 64.  64.]]

  [[ 80.  80.]]

  [[ 96.  96.]]

  [[112. 112.]]

  [[128. 128.]]]]
BD before shift: [[[[ 13.800001  16.199999]]

  [[ 25.8       28.2     ]]

  [[ 37.8       40.2     ]]

  [[ 49.799995  52.200005]]

  [[ 61.799995  64.200005]]

  [[ 73.799995  76.200005]]

  [[ 85.799995  88.200005]]

  [[ 97.8      100.2     ]]]


 [[[ 16.1       18.9     ]]

  [[ 30.099998  32.9     ]]

  [[ 44.1       46.9     ]]

  [[ 58.1       60.9     ]]

  [[ 72.1       74.9     ]]

  [[ 86.1       88.9     ]]

  [[100.1      102.9     ]]

  [[114.100006 116.899994]]]


 [[[ 18.400002  21.599998]]

  [[ 34.4       37.6     ]]

  [[ 50.4       53.6     ]]

  [[ 66.399

In [35]:
import numpy as np

# Define the transformation function
def rel_shift_numpy(x):
    # Append one "column" of zeros to the left
    zero_pad = np.zeros((x.shape[0], 1))
    x_padded = np.concatenate([zero_pad, x], axis=1)

    # Reshape the matrix by shifting the elements in the last dimension
    x_padded = x_padded.reshape(x.shape[1] + 1, x.shape[0], -1)

    # Remove the first "row"
    x_padded = x_padded[1:, :, :]

    # Mask out the upper triangle
    mask = np.triu(np.ones((x_padded.shape[1], x_padded.shape[1])), k=1)
    x_masked = x_padded * (1 - mask[:, :, None])

    return x_masked.transpose(1, 0, 2)

# Create the initial unshifted array based on the provided image
unshifted_array = np.array([
    [0.00, 0.01, 0.02, 0.03],
    [0.10, 0.11, 0.12, 0.13],
    [0.20, 0.21, 0.22, 0.23]
])

# Apply the relative shift
shifted_array = rel_shift_numpy(unshifted_array)

shifted_array

ValueError: operands could not be broadcast together with shapes (4,3,1) (3,3,1) 